<a href="https://colab.research.google.com/github/SergeyHSE/Bagging.github.io/blob/main/RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**1**



In [1]:
#Let's load the load_wine data from sklearn.datasets
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.datasets import load_wine
data = load_wine()

In [2]:
#We will exclude Class 2 objects from the training part.
X, y = data['data'], data['target']

y = np.expand_dims(y, axis=1)

data1 = np.concatenate((X, y), axis=1)
data2 = np.delete(data1, np.where(data1[:, -1] == 2), axis=0)
X_cut = np.delete(data2, 13, axis=1)
y_cut = np.delete(data2, np.s_[0:13], axis=1)
y = np.squeeze(y_cut, axis=-1)

X = pd.DataFrame(data=X_cut, columns=data.feature_names)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((97, 13), (33, 13), (97,), (33,))